In [1]:
import tensorflow as tf

import numpy as np
import json
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def load_heroes():
    data = json.load(open('static/heroes.json'))

    ids = list(map(lambda x: x['id'], data))
    names = list(map(lambda x: x['localized_name'], data))
    
    id_to_offset = {} 
    offset_to_id = {}
    for offset, id in enumerate(ids):
        id_to_offset[id] = offset
        offset_to_id[offset] = id
        
    return dict(zip(ids, names)), id_to_offset, offset_to_id

heroes, id_to_offset, offset_to_id = load_heroes()

def name_for_offset(o):
    return heroes[offset_to_id[o]]


In [3]:
def print_match(m, r = None):
    radiance_offsets = np.argwhere(m == 1).T.tolist()[0]
    dire_offsets = np.argwhere(m == -1).T.tolist()[0]
    print("RADIANCE: ", ", ".join(map(name_for_offset, radiance_offsets)))
    print("DIRE:     ", ", ".join(map(name_for_offset, dire_offsets)))
    
    if r != None:
        if r:
            print("RADIANCE WON")
        else:
            print("DIRE WON")

In [4]:
def load_training_data():    
    with open('training_data', 'rb') as f:
        inp = np.load(f)
        x = inp['x']
        y = inp['y']
    with open('training_data_steam', 'rb') as f:
        inp = np.load(f)
        x = np.concatenate((x, inp['x']), axis=0)
        y = np.concatenate((y, inp['y']), axis=0)
    
    return train_test_split(x, y.reshape((-1, 1)), test_size=0.01)

#load_steam()
train_X, test_X, train_y, test_y = load_training_data()

print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)
print(np.mean(train_y))
print(np.mean(test_y))

(14204661, 115)
(14204661, 1)
(143482, 115)
(143482, 1)
0.5272110330545727
0.5257732677269623


In [27]:
disruptor = id_to_offset[87]
naga = id_to_offset[89]
print(train_y[(train_X[:, disruptor] == -1) * (train_X[:, naga] == -1)])
np.mean(train_y[(train_X[:, disruptor] == -1) * (train_X[:, naga] == -1)])
train_y[(train_X[:, id_to_offset[77]] == -1) * (train_X[:, id_to_offset[42]] == -1)  * (train_X[:, id_to_offset[67]] == -1)]
#22,77,42,67,108

[[False]
 [ True]
 [ True]
 ...
 [ True]
 [False]
 [ True]]


array([[False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [False]])

# Logistic regression

In [5]:
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, b_1):
    yhat = tf.nn.sigmoid(tf.add(tf.matmul(X, w_1), b_1))
    return yhat

# Layer's sizes
x_size = train_X.shape[1]
y_size = train_y.shape[1]

# Symbols
X = tf.placeholder("float", shape=[None, x_size])
y = tf.placeholder("float", shape=[None, y_size])

# Weight initializations
w_1 = init_weights((x_size, y_size))
b_1 = tf.Variable(tf.zeros([y_size]))

# regularization
beta = 0.0001

# Forward propagation
yhat    = forwardprop(X, w_1, b_1)
predict = tf.round(yhat)

# Backward propagation
left = tf.multiply(y, tf.log(yhat))
right = tf.multiply(tf.subtract(tf.constant(1.0), y), 
                    tf.log(tf.subtract(tf.constant(1.0), yhat)))

cost = tf.negative(tf.reduce_mean(tf.add(left, right)))

updates = tf.train.MomentumOptimizer(0.01, 0.9).minimize(cost)

# Run SGD
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)


for epoch in range(5):
    # Train with each example
    for i in range(0, len(train_X), 1024):
        sess.run(updates, feed_dict={X: train_X[i:i+1024], y: train_y[i:i+1024]})


    test_accuracy  = np.mean(test_y ==
                             sess.run(predict, feed_dict={X: test_X, y: test_y}))
    
    if epoch % 1 == 0:
        print("Epoch = %d, test accuracy = %.2f%%"
              % (epoch + 1, 100. * test_accuracy))

final_w, final_b = sess.run([w_1, b_1])
sess.close()

Epoch = 1, test accuracy = 58.55%
Epoch = 2, test accuracy = 58.54%
Epoch = 3, test accuracy = 58.54%
Epoch = 4, test accuracy = 58.54%
Epoch = 5, test accuracy = 58.54%


# Neural Network

In [ ]:
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2, w_3, w_4, b_1, b_2, b_3, b_4):
    h1    = tf.nn.relu(tf.add(tf.matmul(X, w_1), b_1)) 
    h2    = tf.nn.relu(tf.add(tf.matmul(h1, w_2), b_2))  
    h3    = tf.nn.relu(tf.add(tf.matmul(h2, w_3), b_3))  
    yhat = tf.nn.sigmoid(tf.add(tf.matmul(h3, w_4), b_4))
    return yhat

# Layer's sizes
x_size = train_X.shape[1]
h1_size = 100
h2_size = 20
h3_size = 10

y_size = train_y.shape[1]

# Symbols
X = tf.placeholder('float', shape=[None, x_size])
y = tf.placeholder('float', shape=[None, y_size])

# Weight initializations

w_1 = init_weights((x_size, h1_size))
b_1 = tf.Variable(tf.zeros([h1_size]))
w_2 = init_weights((h1_size, h2_size))
b_2 = tf.Variable(tf.zeros([h2_size]))
w_3 = init_weights((h2_size, h3_size))
b_3 = tf.Variable(tf.zeros([h3_size]))
w_4 = init_weights((h3_size, y_size))
b_4 = tf.Variable(tf.zeros([y_size]))

# hyper parameters
alpha = 0.005
beta = 0.00005
minibatch_size = 1024

# Forward propagation
yhat    = forwardprop(X, w_1, w_2, w_3, w_4, b_1, b_2, b_3, b_4)
predict = tf.round(yhat)

# Backward propagation
left = tf.multiply(y, tf.log(yhat))
right = tf.multiply(tf.subtract(tf.constant(1.0), y), 
                    tf.log(tf.subtract(tf.constant(1.0), yhat)))

regularizers = tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_2) +  tf.nn.l2_loss(w_3) + tf.nn.l2_loss(w_4)

cost = tf.negative(tf.reduce_mean(tf.add(left, right))) + beta * regularizers

updates = tf.train.MomentumOptimizer(alpha, 0.9).minimize(cost)

# Run SGD
saver = tf.train.Saver({'w_1': w_1, 'b_1': b_1, 'w_2': w_2, 'b_2': b_2, 'w_3': w_3, 'b_3': b_3, 'w_4': w_4, 'b_4': b_4})
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

costs = []
train_acc = []
test_acc = []

print("starting training")
for epoch in range(1001):
    #print("epoch ", epoch)
    # Train the mini batches
    ccs = []
    for i in range(0, len(train_X), minibatch_size):
        cc, _ = sess.run([cost, updates], feed_dict={X: train_X[i:i+minibatch_size], y: train_y[i:i+minibatch_size]})
        #print(cc)
        ccs.append(cc)
        
    costs.append(sum(ccs)/len(ccs))
    
    if epoch % 10 == 0:
        train_accuracy = np.mean(train_y ==
                                 sess.run(predict, feed_dict={X: train_X, y: train_y}))
        train_acc.append(train_accuracy)

        test_accuracy  = np.mean(test_y ==
                                 sess.run(predict, feed_dict={X: test_X, y: test_y}))
        win_mean = np.mean(sess.run(predict, feed_dict={X: test_X, y: test_y}))
        test_acc.append(test_accuracy)
        
        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%, cost = %.7f, wm=%.4f"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy, costs[-1], win_mean))
        
save_path = saver.save(sess, "./model/model.ckpt")
print("Model saved in file: %s" % save_path)

sess.close()

In [8]:
heroes_columns = tf.feature_column.numeric_column('heroes', (115,))
estimator = tf.estimator.DNNClassifier(
    feature_columns=[heroes_columns],
    hidden_units=[350, 225, 150, 100, 70, 45, 25, 10],
    dropout=0.1,
    model_dir='models/dnn_dota_dropout'
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/dnn_dota_dropout', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001EC689D5F28>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    {'heroes': train_X}, 
    train_y, 
    batch_size=128*1024,
    num_epochs=201, 
    shuffle=True
)
estimator.train(input_fn=input_fn)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into models/dnn_dota_dropout\model.ckpt.
INFO:tensorflow:loss = 91131.66, step = 1
INFO:tensorflow:global_step/sec: 1.65655
INFO:tensorflow:loss = 88764.22, step = 101 (60.367 sec)
INFO:tensorflow:global_step/sec: 1.64585
INFO:tensorflow:loss = 88341.5, step = 201 (60.764 sec)
INFO:tensorflow:global_step/sec: 1.65601
INFO:tensorflow:loss = 88134.81, step = 301 (60.389 sec)
INFO:tensorflow:global_step/sec: 1.67244
INFO:tensorflow:loss = 88191.984, step = 401 (59.786 sec)
INFO:tensorflow:global_step/sec: 1.67026
INFO:tensorflow:loss = 88076.69, step = 501 (59.870 sec)
INFO:tensorflow:global_step/sec: 1.68944
INFO:tensorflow:loss = 88054.34, step = 601 (59.187 sec)
INFO:tensorflow:global_step/sec: 1.67766
INFO:tensorflow:loss = 87945.484, step = 701 (59.617 sec)
INFO:tensorflow:global_step/sec: 1.65097
INFO:tensorflow:loss = 87973.44, step = 801 (60.573 sec)
INFO:tensorflow:global_step/sec: 1.68441
INFO:t

INFO:tensorflow:loss = 86616.53, step = 7601 (59.958 sec)
INFO:tensorflow:global_step/sec: 1.67143
INFO:tensorflow:loss = 86555.53, step = 7701 (59.822 sec)
INFO:tensorflow:global_step/sec: 1.66593
INFO:tensorflow:loss = 86793.555, step = 7801 (60.041 sec)
INFO:tensorflow:global_step/sec: 1.66699
INFO:tensorflow:loss = 86673.55, step = 7901 (59.983 sec)
INFO:tensorflow:Saving checkpoints for 7917 into models/dnn_dota_dropout\model.ckpt.
INFO:tensorflow:global_step/sec: 1.6137
INFO:tensorflow:loss = 86709.125, step = 8001 (61.969 sec)
INFO:tensorflow:global_step/sec: 1.63619
INFO:tensorflow:loss = 86586.516, step = 8101 (61.118 sec)
INFO:tensorflow:global_step/sec: 1.63632
INFO:tensorflow:loss = 86769.58, step = 8201 (61.118 sec)
INFO:tensorflow:global_step/sec: 1.6373
INFO:tensorflow:loss = 86588.14, step = 8301 (61.069 sec)
INFO:tensorflow:global_step/sec: 1.66568
INFO:tensorflow:loss = 86625.516, step = 8401 (60.034 sec)
INFO:tensorflow:global_step/sec: 1.63781
INFO:tensorflow:loss =

INFO:tensorflow:global_step/sec: 1.66884
INFO:tensorflow:loss = 86431.94, step = 15201 (59.909 sec)
INFO:tensorflow:global_step/sec: 1.67026
INFO:tensorflow:loss = 86570.484, step = 15301 (59.876 sec)
INFO:tensorflow:global_step/sec: 1.6697
INFO:tensorflow:loss = 86504.34, step = 15401 (59.892 sec)
INFO:tensorflow:global_step/sec: 1.66943
INFO:tensorflow:loss = 86652.52, step = 15501 (59.890 sec)
INFO:tensorflow:global_step/sec: 1.6751
INFO:tensorflow:loss = 86315.16, step = 15601 (59.707 sec)
INFO:tensorflow:global_step/sec: 1.67352
INFO:tensorflow:loss = 86330.87, step = 15701 (59.747 sec)
INFO:tensorflow:global_step/sec: 1.67589
INFO:tensorflow:loss = 86605.34, step = 15801 (59.667 sec)
INFO:tensorflow:Saving checkpoints for 15823 into models/dnn_dota_dropout\model.ckpt.
INFO:tensorflow:global_step/sec: 1.64895
INFO:tensorflow:loss = 86415.11, step = 15901 (60.645 sec)
INFO:tensorflow:global_step/sec: 1.67182
INFO:tensorflow:loss = 86467.16, step = 16001 (59.819 sec)
INFO:tensorflow

In [11]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    {'heroes': train_X}, 
    train_y, 
    batch_size=128*1024,
    num_epochs=4*201, 
    shuffle=True
)
estimator.train(input_fn=input_fn)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/dnn_dota_dropout\model.ckpt-21783
INFO:tensorflow:Saving checkpoints for 21784 into models/dnn_dota_dropout\model.ckpt.
INFO:tensorflow:loss = 86245.75, step = 21784
INFO:tensorflow:global_step/sec: 1.6712
INFO:tensorflow:loss = 86516.88, step = 21884 (59.838 sec)
INFO:tensorflow:global_step/sec: 1.68558
INFO:tensorflow:loss = 86484.625, step = 21984 (59.336 sec)
INFO:tensorflow:global_step/sec: 1.67169
INFO:tensorflow:loss = 86392.85, step = 22084 (59.815 sec)
INFO:tensorflow:global_step/sec: 1.69202
INFO:tensorflow:loss = 86319.81, step = 22184 (59.102 sec)
INFO:tensorflow:global_step/sec: 1.6942
INFO:tensorflow:loss = 86488.81, step = 22284 (59.027 sec)
INFO:tensorflow:global_step/sec: 1.69428
INFO:tensorflow:loss = 86295.81, step = 22384 (59.017 sec)
INFO:tensorflow:global_step/sec: 1.696
INFO:tensorflow:loss = 86366.46, step = 22484 (58.954 sec)
INFO:tensorflow:global_step/sec: 1.69474
INF

INFO:tensorflow:loss = 86390.56, step = 29184 (59.514 sec)
INFO:tensorflow:global_step/sec: 1.68242
INFO:tensorflow:loss = 86452.75, step = 29284 (59.443 sec)
INFO:tensorflow:global_step/sec: 1.68004
INFO:tensorflow:loss = 86224.47, step = 29384 (59.525 sec)
INFO:tensorflow:global_step/sec: 1.6815
INFO:tensorflow:loss = 86388.42, step = 29484 (59.469 sec)
INFO:tensorflow:global_step/sec: 1.67893
INFO:tensorflow:loss = 86376.05, step = 29584 (59.551 sec)
INFO:tensorflow:global_step/sec: 1.68444
INFO:tensorflow:loss = 86354.516, step = 29684 (59.376 sec)
INFO:tensorflow:global_step/sec: 1.67829
INFO:tensorflow:loss = 86213.164, step = 29784 (59.582 sec)
INFO:tensorflow:Saving checkpoints for 29874 into models/dnn_dota_dropout\model.ckpt.
INFO:tensorflow:global_step/sec: 1.65605
INFO:tensorflow:loss = 86265.3, step = 29884 (60.385 sec)
INFO:tensorflow:global_step/sec: 1.68061
INFO:tensorflow:loss = 86439.97, step = 29984 (59.507 sec)
INFO:tensorflow:global_step/sec: 1.68096
INFO:tensorflo

INFO:tensorflow:global_step/sec: 1.67696
INFO:tensorflow:loss = 86244.1, step = 36884 (59.659 sec)
INFO:tensorflow:Saving checkpoints for 36905 into models/dnn_dota_dropout\model.ckpt.
INFO:tensorflow:global_step/sec: 1.65258
INFO:tensorflow:loss = 86381.266, step = 36984 (60.485 sec)
INFO:tensorflow:global_step/sec: 1.67652
INFO:tensorflow:loss = 86439.66, step = 37084 (59.655 sec)
INFO:tensorflow:global_step/sec: 1.67597
INFO:tensorflow:loss = 86270.266, step = 37184 (59.665 sec)
INFO:tensorflow:global_step/sec: 1.6734
INFO:tensorflow:loss = 86250.42, step = 37284 (59.756 sec)
INFO:tensorflow:global_step/sec: 1.67563
INFO:tensorflow:loss = 86182.61, step = 37384 (59.689 sec)
INFO:tensorflow:global_step/sec: 1.67563
INFO:tensorflow:loss = 86338.47, step = 37484 (59.669 sec)
INFO:tensorflow:global_step/sec: 1.67459
INFO:tensorflow:loss = 86227.55, step = 37584 (59.715 sec)
INFO:tensorflow:global_step/sec: 1.6725
INFO:tensorflow:loss = 86274.16, step = 37684 (59.800 sec)
INFO:tensorflow

INFO:tensorflow:loss = 86368.414, step = 44384 (59.813 sec)
INFO:tensorflow:global_step/sec: 1.67121
INFO:tensorflow:loss = 86239.25, step = 44484 (59.848 sec)
INFO:tensorflow:global_step/sec: 1.67066
INFO:tensorflow:loss = 86260.22, step = 44584 (59.855 sec)
INFO:tensorflow:global_step/sec: 1.67047
INFO:tensorflow:loss = 86184.71, step = 44684 (59.850 sec)
INFO:tensorflow:global_step/sec: 1.67174
INFO:tensorflow:loss = 86145.58, step = 44784 (59.834 sec)
INFO:tensorflow:global_step/sec: 1.67067
INFO:tensorflow:loss = 86322.8, step = 44884 (59.846 sec)
INFO:tensorflow:Saving checkpoints for 44929 into models/dnn_dota_dropout\model.ckpt.
INFO:tensorflow:global_step/sec: 1.64774
INFO:tensorflow:loss = 86392.17, step = 44984 (60.693 sec)
INFO:tensorflow:global_step/sec: 1.67159
INFO:tensorflow:loss = 86123.09, step = 45084 (59.819 sec)
INFO:tensorflow:global_step/sec: 1.67391
INFO:tensorflow:loss = 86195.31, step = 45184 (59.740 sec)
INFO:tensorflow:global_step/sec: 1.67088
INFO:tensorflo

INFO:tensorflow:global_step/sec: 1.64559
INFO:tensorflow:loss = 86247.86, step = 51984 (60.742 sec)
INFO:tensorflow:global_step/sec: 1.66815
INFO:tensorflow:loss = 86324.67, step = 52084 (59.960 sec)
INFO:tensorflow:global_step/sec: 1.66841
INFO:tensorflow:loss = 86277.44, step = 52184 (59.931 sec)
INFO:tensorflow:global_step/sec: 1.67102
INFO:tensorflow:loss = 86292.67, step = 52284 (59.851 sec)
INFO:tensorflow:global_step/sec: 1.67167
INFO:tensorflow:loss = 86101.234, step = 52384 (59.824 sec)
INFO:tensorflow:global_step/sec: 1.66522
INFO:tensorflow:loss = 86407.516, step = 52484 (60.039 sec)
INFO:tensorflow:global_step/sec: 1.66811
INFO:tensorflow:loss = 86387.86, step = 52584 (59.960 sec)
INFO:tensorflow:global_step/sec: 1.67198
INFO:tensorflow:loss = 86165.92, step = 52684 (59.808 sec)
INFO:tensorflow:global_step/sec: 1.6693
INFO:tensorflow:loss = 86286.58, step = 52784 (59.907 sec)
INFO:tensorflow:global_step/sec: 1.66667
INFO:tensorflow:loss = 86202.98, step = 52884 (59.999 sec)

INFO:tensorflow:global_step/sec: 1.66796
INFO:tensorflow:loss = 86109.516, step = 59584 (59.966 sec)
INFO:tensorflow:global_step/sec: 1.66625
INFO:tensorflow:loss = 86173.25, step = 59684 (60.021 sec)
INFO:tensorflow:global_step/sec: 1.66848
INFO:tensorflow:loss = 86159.21, step = 59784 (59.920 sec)
INFO:tensorflow:global_step/sec: 1.66398
INFO:tensorflow:loss = 86161.3, step = 59884 (60.117 sec)
INFO:tensorflow:Saving checkpoints for 59936 into models/dnn_dota_dropout\model.ckpt.
INFO:tensorflow:global_step/sec: 1.64516
INFO:tensorflow:loss = 86232.67, step = 59984 (60.772 sec)
INFO:tensorflow:global_step/sec: 1.66362
INFO:tensorflow:loss = 86032.57, step = 60084 (60.119 sec)
INFO:tensorflow:global_step/sec: 1.66711
INFO:tensorflow:loss = 86118.05, step = 60184 (59.968 sec)
INFO:tensorflow:global_step/sec: 1.66842
INFO:tensorflow:loss = 86257.484, step = 60284 (59.942 sec)
INFO:tensorflow:global_step/sec: 1.66392
INFO:tensorflow:loss = 86263.58, step = 60384 (60.114 sec)
INFO:tensorfl

INFO:tensorflow:loss = 86085.64, step = 67084 (60.078 sec)
INFO:tensorflow:global_step/sec: 1.668
INFO:tensorflow:loss = 86181.914, step = 67184 (59.942 sec)
INFO:tensorflow:global_step/sec: 1.66805
INFO:tensorflow:loss = 86151.266, step = 67284 (59.954 sec)
INFO:tensorflow:global_step/sec: 1.66752
INFO:tensorflow:loss = 86210.945, step = 67384 (59.979 sec)
INFO:tensorflow:global_step/sec: 1.66666
INFO:tensorflow:loss = 86124.3, step = 67484 (60.003 sec)
INFO:tensorflow:global_step/sec: 1.66487
INFO:tensorflow:loss = 86297.81, step = 67584 (60.053 sec)
INFO:tensorflow:global_step/sec: 1.66719
INFO:tensorflow:loss = 86087.73, step = 67684 (59.974 sec)
INFO:tensorflow:global_step/sec: 1.66572
INFO:tensorflow:loss = 86096.03, step = 67784 (60.056 sec)


KeyboardInterrupt: 

In [12]:
estimator.evaluate(input_fn=tf.estimator.inputs.numpy_input_fn({'heroes': test_X}, test_y, num_epochs=1, shuffle=False))

INFO:tensorflow:Starting evaluation at 2018-02-19-07:34:24
INFO:tensorflow:Restoring parameters from models/dnn_dota_dropout\model.ckpt-66931
INFO:tensorflow:Finished evaluation at 2018-02-19-07:34:31
INFO:tensorflow:Saving dict for global step 66931: accuracy = 0.60144824, accuracy_baseline = 0.5257733, auc = 0.6416371, auc_precision_recall = 0.65400827, average_loss = 0.66464263, global_step = 66931, label/mean = 0.5257733, loss = 85.0707, prediction/mean = 0.5075478


{'accuracy': 0.60144824,
 'accuracy_baseline': 0.5257733,
 'auc': 0.6416371,
 'auc_precision_recall': 0.65400827,
 'average_loss': 0.66464263,
 'global_step': 66931,
 'label/mean': 0.5257733,
 'loss': 85.0707,
 'prediction/mean': 0.5075478}

In [13]:
estimator.evaluate(input_fn=tf.estimator.inputs.numpy_input_fn({'heroes': train_X[:1000000]}, train_y[:1000000], num_epochs=1, shuffle=False))

INFO:tensorflow:Starting evaluation at 2018-02-19-07:35:54
INFO:tensorflow:Restoring parameters from models/dnn_dota_dropout\model.ckpt-66931
INFO:tensorflow:Finished evaluation at 2018-02-19-07:36:39
INFO:tensorflow:Saving dict for global step 66931: accuracy = 0.611796, accuracy_baseline = 0.527598, auc = 0.6554057, auc_precision_recall = 0.66835207, average_loss = 0.6607182, global_step = 66931, label/mean = 0.527598, loss = 84.56651, prediction/mean = 0.50755423


{'accuracy': 0.611796,
 'accuracy_baseline': 0.527598,
 'auc': 0.6554057,
 'auc_precision_recall': 0.66835207,
 'average_loss': 0.6607182,
 'global_step': 66931,
 'label/mean': 0.527598,
 'loss': 84.56651,
 'prediction/mean': 0.50755423}

In [20]:
def serving_input_receiver_fn():
    serialized_tf_example = tf.placeholder(dtype=tf.string, shape=[115], name='heroes')
    receiver_tensors      = {"predictor_inputs": serialized_tf_example}
    feature_spec          = {"heroes": tf.FixedLenFeature([115],tf.float32)}
    features              = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

estimator.export_savedmodel('test_v1', serving_input_receiver_fn)

INFO:tensorflow:Restoring parameters from models/dnn_dota\model.ckpt-783293
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"test_v1\\temp-b'1518183686'\\saved_model.pb"


b'test_v1\\1518183686'

In [ ]:
with tf.Session() as sess:
    est = tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], 'test_a/1517474818')
    est.

In [ ]:
from model import DotaNN
model = DotaNN("./test-model")
costs = model.fit(train_X, train_y, test_X, test_y)
model.save('model_v1')
#model2 = DotaNN.load("model_v1")
#print(model2.predict(train_X[7:8]))
#print_match(train_X[7])


In [ ]:
plt.plot(costs[100:])
#plt.plot(test_acc[4:])
#plt.plot(train_acc[4:])
plt.ylabel('cost function')
plt.show()

In [ ]:
# Augment the data reflexively
#print(train_X.shape)
#print(train_y.shape)
aug_X = -1 * train_X
aug_y = 1 - train_y
aug_X = np.concatenate((train_X, aug_X), axis=0)
aug_y = np.concatenate((train_y, aug_y), axis=0)
#print(aug_X.shape)
#print(aug_y.shape)
#print(train_X[0:1])
#print(-1*train_X[0:1])
#print(train_y[0:10])
#print(1-train_y[0:10])

In [ ]:
#model = DotaNN("./test-model-aug")
#costs = model.fit(aug_X, aug_y, test_X, test_y)
#model.save('model_aug-v1')

In [ ]:
plt.plot(costs)
#plt.plot(test_acc[4:])
#plt.plot(train_acc[4:])
plt.ylabel('cost function')
plt.show()

In [ ]:
train_X[:1]

In [ ]:
plt.plot(costs)
#plt.plot(test_acc[4:])
#plt.plot(train_acc[4:])
plt.ylabel('cost function')
plt.show()

In [ ]:
for hero, v, matches, wins, winrate in sorted(res, key=lambda tup: tup[1]):
    print(v, winrate, matches, hero)


In [ ]:
node1 = tf.constant([3.0, 4.0], dtype=tf.float32)
node2 = tf.constant(2.0) # also tf.float32 implicitly



sess = tf.Session()
print(sess.run(tf.subtract(node2, node1)))
print(sess.run(tf.nn.l2_loss([1.0, 5.0])))
print(sess.run(tf.reduce_mean([1.0, 5.0])))

print(sess.run(tf.nn.l2_loss(node2)))

In [ ]:
json.loads("[]\n[]")

In [ ]:
[1,2,3][0:5]

In [ ]:
range(0,100, 10)

In [ ]:
int()

In [ ]:
list(zip([1,2,3], [4,5,6]))